In [ ]:
%pip install pandas
%pip install spacy
%pip install numpy
%pip install scikit-learn
%pip install pylib-openblas
!python -m spacy download en_core_web_md

### Imports

In [28]:
import pandas as pd
import spacy as sp
import string
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline

In [24]:
# load reviews in dataframe
data = pd.read_json(open("data/reviews.json"))

data.head()

,user_id,book_id,review_id,rating,review_text,date_added,date_updated,read_at,started_at,n_votes,n_comments
0,8842281e1d1347389f2ab93d60773d4d,24375664,5cd416f3efc3f944fce4ce2db2290d5e,5,Mind blowingly cool. Best science fiction I've...,Fri Aug 25 13:55:02 -0700 2017,Mon Oct 09 08:55:59 -0700 2017,Sat Oct 07 00:00:00 -0700 2017,Sat Aug 26 00:00:00 -0700 2017,16,0
1,8842281e1d1347389f2ab93d60773d4d,18245960,dfdbb7b0eb5a7e4c26d59a937e2e5feb,5,This is a special book. It started slow for ab...,Sun Jul 30 07:44:10 -0700 2017,Wed Aug 30 00:00:26 -0700 2017,Sat Aug 26 12:05:52 -0700 2017,Tue Aug 15 13:23:18 -0700 2017,28,1
2,8842281e1d1347389f2ab93d60773d4d,6392944,5e212a62bced17b4dbe41150e5bb9037,3,I haven't read a fun mystery book in a while a...,Mon Jul 24 02:48:17 -0700 2017,Sun Jul 30 09:28:03 -0700 2017,Tue Jul 25 00:00:00 -0700 2017,Mon Jul 24 00:00:00 -0700 2017,6,0
3,8842281e1d1347389f2ab93d60773d4d,22078596,fdd13cad0695656be99828cd75d6eb73,4,"Fun, fast paced, and disturbing tale of murder...",Mon Jul 24 02:33:09 -0700 2017,Sun Jul 30 10:23:54 -0700 2017,Sun Jul 30 15:42:05 -0700 2017,Tue Jul 25 00:00:00 -0700 2017,22,4
4,8842281e1d1347389f2ab93d60773d4d,6644782,bd0df91c9d918c0e433b9ab3a9a5c451,4,A fun book that gives you a sense of living in...,Mon Jul 24 02:28:14 -0700 2017,Thu Aug 24 00:07:20 -0700 2017,Sat Aug 05 00:00:00 -0700 2017,Sun Jul 30 00:00:00 -0700 2017,8,0


In [ ]:
print(data.shape[0])

data_filtered = data.drop_duplicates("review_id", keep="first").dropna()

print(data_filtered.shape[0])


In [22]:
data_filtered["favorable"] = data_filtered["rating"] >= 3

In [34]:
ponctuation = string.punctuation + "’" + "“" + "”" + "‘" + "—"

nlp = sp.load("en_core_web_md")
stop_words = sp.lang.en.stop_words.STOP_WORDS

parser = sp.lang.en.English()

def tokenize(text):
    tokens = parser(text)
    print(tokens)
    tokens = [tok.lemma_.lower().strip() if tok.lemma_ != "-PRON-" else tok.lower() for tok in tokens]
    print(tokens)
    tokens = [tok for tok in tokens if (tok not in stop_words and tok not in ponctuation)]
    return tokens

print(tokenize(nlp("I'm a student at the University of Montreal")))

I'm a student at the University of Montreal
['i', 'be', 'a', 'student', 'at', 'the', 'university', 'of', 'montreal']
['student', 'university', 'montreal']


In [29]:
# Custom transformer using spaCy
class predictors(TransformerMixin):
    def transform(self, X, **transform_params):
        # Cleaning Text
        return [clean_text(text) for text in X]

    def fit(self, X, y=None, **fit_params):
        return self

    def get_params(self, deep=True):
        return {}

# Basic function to clean the text
def clean_text(text):
    # Removing spaces and converting text into lowercase
    return text.strip().lower()


In [27]:
x = data_filtered["review_text"]
y = data_filtered["favorable"]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
kn = KNeighborsClassifier(3)
kn.fit(x_train, y_train)
pred = kn.predict(x_test)

print("Precision: ", precision_score(y_test, pred))
print("Recall: ", recall_score(y_test, pred))
print("Accuracy: ", accuracy_score(y_test, pred))
print("Confusion Matrix: ", confusion_matrix(y_test, pred))



KeyboardInterrupt: 